In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%tensorflow_version 1.x
!pip install --upgrade h5py==2.10.0
!pip install opencv-python
!pip install Shapely

import sys
import shutil
import os
import cv2
import numpy as np

!wget "https://www.dropbox.com/s/z8ux8vw2l2cmq7k/Mask_RCNN_withRW.zip"
!unzip Mask_RCNN_withRW.zip

sys.path.append("/content/Mask_RCNN_withRW/mrcnn")
from m_rcnn import *
%matplotlib inline

TensorFlow 1.x selected.
     |████████████████████████████████| 2.9 MB 5.5 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
--2022-02-14 11:36:36--  https://www.dropbox.com/s/z8ux8vw2l2cmq7k/Mask_RCNN_withRW.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/z8ux8vw2l2cmq7k/Mask_RCNN_withRW.zip [following]
--2022-02-14 11:36:36--  https://www.dropbox.com/s/raw/z8ux8vw2l2cmq7k/Mask_RCNN_withRW.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc14c65df27c043c7bb903fa820.dl.dropboxusercontent.com/cd/0/inline/BfsyG6OxxXEgzNveqdxUjWJg7c6aysYUwQj9dLqeyCK7Fkw51DFu-bEmzT-e3uhbwfXJpvXCq2SD6xvoiBQJue02uPDlTjZ9qoePnpr-TxQPqCg

Using TensorFlow backend.


In [3]:
!wget "https://www.dropbox.com/s/evfvlt3eitqpnkr/mask_rcnn_object_0005.h5"

--2022-02-14 11:36:47--  https://www.dropbox.com/s/evfvlt3eitqpnkr/mask_rcnn_object_0005.h5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/evfvlt3eitqpnkr/mask_rcnn_object_0005.h5 [following]
--2022-02-14 11:36:47--  https://www.dropbox.com/s/raw/evfvlt3eitqpnkr/mask_rcnn_object_0005.h5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9b11bebd4ea2de75857a0b555d.dl.dropboxusercontent.com/cd/0/inline/Bfu25aeFf2avjhi-acPbAs_9sHAsritmbunJNG0uSAfjB7tY4zVRjFVofhkupnD64E7f2b_BZhdq8LNSsVwypGdDMP04r90sFIYd6IoUr0JXCw863naUoPZVPhm72o2hNORTUnKzDuDQfgRAjWzrJR9_/file# [following]
--2022-02-14 11:36:47--  https://uc9b11bebd4ea2de75857a0b555d.dl.dropboxusercontent.com/cd/0/inline/Bfu25aeFf2avjhi-acPbAs_9sHAsritmbunJNG0uSAfjB7tY4zVRjF

In [4]:
from visualize import random_colors, get_mask_contours, draw_mask

In [5]:
list = os.listdir("/content/drive/MyDrive/TER-final/testImages")
number_files = len(list)
print(number_files)

51


In [6]:
loc = "/content/drive/MyDrive/TER-final/testImages" 

names=[] 

for images in os.listdir(loc): 
    file = loc+"/"+images
    filestr = file.split("/")
    names.append(filestr[6])

In [7]:
# Load Image
test_model, inference_config = load_inference_model(1, "/content/mask_rcnn_object_0005.h5")

for images in range(number_files):

    print("=========================")
    print("Processing image: ", images)
    
    img = cv2.imread("/content/drive/MyDrive/TER-final/testImages/"+names[images])
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect results
    r = test_model.detect([image])[0]
    colors = random_colors(80)
    
    ## Save picture with colored mask in the folder "outputImagesWithMasksColors"
    object_count = len(r["class_ids"])

    
    for i in range(object_count):
      # 1. Mask
      mask = r["masks"][:, :, i]
      contours = get_mask_contours(mask)

      for cnt in contours:
          cv2.polylines(img, [cnt], True, colors[i], 2)
          img = draw_mask(img, [cnt], colors[i])

    cv2.imwrite(os.path.join("/content/drive/MyDrive/TER-final/outputImagesWithMasksColors" , names[images]), img)
    
    ## Create the mask in black and white
    height, width, channels = image.shape
    image_shape = height, width
    
    ## We want a black and white image to pass to the annotation script
    black_image = np.zeros((height,width,3), np.uint8)
    white = (255, 255, 255)
    
    ## Save the mask in B&W in the folder "outputImagesWithMasksBlackAndWhite"
    object_count = len(r["class_ids"])
    
    for i in range(object_count):
      # 1. Mask
      mask = r["masks"][:, :, i]
      contours = get_mask_contours(mask)
      for cnt in contours:

          black_image = draw_mask(black_image, [cnt], white)
          cv2.polylines(black_image, [cnt], True, white, 2)
          cv2.fillPoly(black_image, [cnt], white)
          black_image = draw_mask(black_image, [cnt], white)
          cv2.polylines(black_image, [cnt], True, white, 2)
    
    imgAux = names[images].split(".")
    cv2.imwrite(os.path.join("/content/drive/MyDrive/TER-final/outputImagesWithMasksBlackAndWhite" , imgAux[0]+".png"), black_image)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights from  /content/mask_rcnn_object_0005.h5
Processing image:  0

Processing image:  1
Processing image:  2
Processing image:  3
Processing image:  4
Processing image:  5
Processing image:  6
Processing image:  7
Processing image:  8
Processing image:  9
Processing image:  10
Processing image:  11
Processing image:  12
Processing image:  13
Processing image:  14
Processing image:  15
Processing image:  16
Processing image:  17
Processing image:  18
Processing image:  19
Processing image:  20
Processing image:  21
Processing image:  22
Processing image:  23
Processing image:  24
Processing image:  25
Processing image:  26
Processing image:  27
Processing image:  28
Processing i

In [8]:
from PIL import Image
import numpy as np
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon 
import os
import json

def create_sub_masks(mask_image, width, height):
    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]

            # Check to see if we have created a sub-mask...
            pixel_str = str(pixel)
            sub_mask = sub_masks.get(pixel_str)
            if sub_mask is None:
               # Create a sub-mask (one bit per pixel) and add to the dictionary
                # Note: we add 1 pixel of padding in each direction
                # because the contours module doesn"t handle cases
                # where pixels bleed to the edge of the image
                sub_masks[pixel_str] = Image.new("1", (width+2, height+2))

            # Set the pixel value to 1 (default is 0), accounting for padding
            sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

def create_sub_mask_annotation(sub_mask):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(np.array(sub_mask), 0.5, positive_orientation="low")

    polygons = []
    segmentations = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        
        if(poly.is_empty):
            # Go to next iteration, dont save empty values in list
            continue

        polygons.append(poly)

        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)
    
    return polygons, segmentations

def create_category_annotation(category_dict):
    category_list = []

    for key, value in category_dict.items():
        category = {
            "supercategory": key,
            "id": value,
            "name": key
        }
        category_list.append(category)

    return category_list

def create_image_annotation(file_name, width, height, image_id):
    images = {
        "file_name": file_name,
        "height": height,
        "width": width,
        "id": image_id
    }

    return images

def create_annotation_format(polygon, segmentation, image_id, category_id, annotation_id):
    min_x, min_y, max_x, max_y = polygon.bounds
    width = max_x - min_x
    height = max_y - min_y
    bbox = (min_x, min_y, width, height)
    area = polygon.area

    annotation = {
        "segmentation": segmentation,
        "area": area,
        "iscrowd": 0,
        "image_id": image_id,
        "bbox": bbox,
        "category_id": category_id,
        "id": annotation_id
    }

    return annotation

def get_coco_json_format():
    # Standard COCO format 
    coco_format = {
        "info": {},
        "licenses": [],
        "images": [{}],
        "categories": [{}],
        "annotations": [{}]
    }

    return coco_format

In [9]:
import glob

# Label ids of the dataset
category_ids = {
    "fish": 1
}

# Define which colors match which categories in the images
category_colors = {
    "(255, 255, 255)": 1 # Fish
}

# Define the ids that are a multiplolygon. In our case: wall, roof and sky
multipolygon_ids = [2, 5, 6]

# Get "images" and "annotations" info 
def images_annotations_info(maskpath):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []
    
    n_images = len(glob.glob(maskpath + "*.png"))
    print("n_images", n_images)
    cont = 0
    for mask_image in glob.glob(maskpath + "*.png"):
        cont = cont + 1
        print("=========================")
        print("Image", cont, "of", n_images)
        # The mask image is *.jpg but the original image is *.png.
        # We make a reference to the original file in the COCO JSON file
        original_file_name = os.path.basename(mask_image).split(".")[0] + ".jpg"
        
        print("name", original_file_name)

        # Open the image and (to be sure) we convert it to RGB
        mask_image_open = Image.open(mask_image).convert("RGB")
        w, h = mask_image_open.size
        
        # "images" info 
        image = create_image_annotation(original_file_name, w, h, image_id)
        images.append(image)

        sub_masks = create_sub_masks(mask_image_open, w, h)
        for color, sub_mask in sub_masks.items():
           
            if color == '(255, 255, 255)':
                category_id = category_colors[color]

                # "annotations" info
                polygons, segmentations = create_sub_mask_annotation(sub_mask)

                # Check if we have classes that are a multipolygon
                if category_id in multipolygon_ids:
                    # Combine the polygons to calculate the bounding box and area
                    multi_poly = MultiPolygon(polygons)
                                
                    annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)

                    annotations.append(annotation)
                    annotation_id += 1
                else:
                    for i in range(len(polygons)):
                        # Cleaner to recalculate this variable
                        segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    
                        annotation = create_annotation_format(polygons[i], segmentation, image_id, category_id, annotation_id)
                    
                        annotations.append(annotation)
                        annotation_id += 1
        image_id += 1              
                
    return images, annotations, annotation_id

if __name__ == "__main__":
    # Get the standard COCO JSON format
    coco_format = get_coco_json_format()
    
    for keyword in ["annotations"]:
        mask_path = "/content/drive/MyDrive/TER-final/outputImagesWithMasksBlackAndWhite/"
        
        # Create category section
        coco_format["categories"] = create_category_annotation(category_ids)
    
        # Create images and annotations sections
        coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

        with open("/content/drive/MyDrive/TER-final/testAnnotations/testAnnotations.json","w") as outfile:
            json.dump(coco_format, outfile)
        
        print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

n_images 51
Image 1 of 51
name 9852_Acanthopagrus_palmaris_f000011.jpg
Image 2 of 51
name 9852_Acanthopagrus_palmaris_f000017.jpg
Image 3 of 51
name 9852_Acanthopagrus_palmaris_f000039.jpg
Image 4 of 51
name 9852_Acanthopagrus_palmaris_f000029.jpg
Image 5 of 51
name 9852_Acanthopagrus_palmaris_f000013.jpg
Image 6 of 51
name 9852_Acanthopagrus_palmaris_f000038.jpg
Image 7 of 51
name 9852_Acanthopagrus_palmaris_f000048.jpg
Image 8 of 51
name 9852_Acanthopagrus_palmaris_f000012.jpg
Image 9 of 51
name 9852_Acanthopagrus_palmaris_f000016.jpg
Image 10 of 51
name 9852_Acanthopagrus_palmaris_f000028.jpg
Image 11 of 51
name 9852_Acanthopagrus_palmaris_f000014.jpg
Image 12 of 51
name 9852_Acanthopagrus_palmaris_f000015.jpg
Image 13 of 51
name 9852_Acanthopagrus_palmaris_f000005.jpg
Image 14 of 51
name 9852_Acanthopagrus_palmaris_f000007.jpg
Image 15 of 51
name 9852_Acanthopagrus_palmaris_f000004.jpg
Image 16 of 51
name 9852_Acanthopagrus_palmaris_f000006.jpg
Image 17 of 51
name 9852_Acanthopagru

In [ ]:
## now we can delete the images contained in testImages folder (not the folder itself) because they became part of the trainImages
!rm -r "/content/drive/MyDrive/TER-final/testImages/"
!mkdir "/content/drive/MyDrive/TER-final/testImages"
